## Benchmarking

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Benchmarking import *

/home/arvindsk/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
user = "arvindsk"
# user = "wknd37"

datasets_benchmark = [
    
# # full set of full-unbal_unbal
# "/home/" + user + "/xgmix_expts/benchmark_data/full/unbal_unbal/latino/",
# "/home/" + user + "/xgmix_expts/benchmark_data/full/unbal_unbal/five/",
# "/home/" + user + "/xgmix_expts/benchmark_data/full/unbal_unbal/seven/",

# # full set of full-bal_admix
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_admix/latino/",
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_admix/five/",
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_admix/seven/",

# UKB
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/unbal_unbal/latino/",
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/unbal_unbal/five/",
"/home/" + user + "/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/",
    
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_admix/latino/",
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_admix/five/",
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_admix/seven/",

# # full-bal_unbal only for seven and latino - this way we complete a latino sweep to compare different balances
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_unbal/seven/",
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_unbal/latino/",
# # full-bal_bal only for latino - seems like a nice expt
# "/home/" + user + "/xgmix_expts/benchmark_data/full/bal_bal/latino/",

# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_unbal/seven/",
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_unbal/latino/",
# "/home/" + user + "/xgmix_expts/benchmark_data/ukb/bal_bal/latino/"

]


In [4]:
def run_bm(base, smooth, chm, gen_expts, models_exist=False, local_metric=None):

    for data_path in datasets_benchmark:

        # data_path must end with generated_data
        data_path = data_path + "generated_data/"

        for gen_name in gen_expts:

            bm_root = data_path + "chm{}/{}/".format(chm,gen_name)
            snp, bal, dat = np.array(bm_root.split("/"))[-7:-4]
            
            if snp == "full":
                W = 1000
            elif snp== "ukb":
                W = 30
            
            metric_path = bm_root + "test_sk_benchmark_results.pkl"
            if local_metric is not None:
                metric_path = metric_path.replace("arvindsk", local_metric)

            gens = gen_expts[gen_name]

            print("-"*100)
            print("Experiment details")
            print("Dataset used: {}".format(data_path))
            print("Generations used: {}".format(gens))
            print("Metrics will come up at: {}".format(metric_path))

            metrics = bm_train(base, smooth, 
                               bm_root, data_path, 
                               gens, chm=chm,
                               W=W, 
                               load_base=True, load_smooth=True, 
                               eval=False,
                               models_exist=models_exist,
                               verbose=True,
                               only_founders=True,# FOR STRING KERNEL
                               quick_eval=True
                              )

            save_dict(metrics, metric_path)
            

In [ ]:
base = ["svm_random_string_kernel"]
smooth = ["xgb"]

# here we run 3 base, 3 smooth, 2 datasets, 3 populations, 2 balance

chm = 20
gen_expts = {
    "all" : [0,2,4,8,16,24,32,48,64,72,100]
}
run_bm(base, smooth, chm, gen_expts, models_exist=False, local_metric="arvindsk")

----------------------------------------------------------------------------------------------------
Experiment details
Dataset used: /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/
Generations used: [0, 2, 4, 8, 16, 24, 32, 48, 64, 72, 100]
Metrics will come up at: /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/chm20/all/test_sk_benchmark_results.pkl
Reading data...
BASE: svm_random_string_kernel
/home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/chm20/all/base_models/svm_random_string_kernel.pkl
SMOOTH: xgb
/home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/chm20/all/models/svm_random_string_kernel_xgb.pkl
Quick Evaluation...


In [ ]:
base = ["rf","lgb", "xgb", "logreg"]
smooth = ["xgb", "crf", "cnv"]
for gen_name in gen_expts:
    for data_path in datasets_benchmark:
        bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
        print(bm_root)
        metric_path = bm_root + "benchmark_results.pkl"
        M = load_dict(metric_path)
        accs = plt_metric(base ,smooth, M, "val_acc")

In [ ]:
import seaborn as sns; 
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

def get_metric(base,smooth,Metrics,metric):
    M = {}
    for i, b in enumerate(base):
        M[b] = {}
        for j, s in enumerate(smooth):
            M[b][s] = Metrics[b+"_"+s][metric]
            
    return pd.DataFrame(M)


def plt_metric(base,smooth,Metrics,metric,vminmax=(None,None),cbar=True):
    
    if metric == "gen_performance":
        cmap = { 0:'k', 1:'b',  2:'y', 3:'g'}
        lmap = { 0:'-', 1:'--', 2:':', 3:'-.' }
        v_accs = get_metric(base, smooth, Metrics, "val_acc")
        for i, b in enumerate(base):
            for j, s in enumerate(smooth):
                m_name = b + "_" + s
                data = Metrics[m_name][metric] 
                plt.plot(data["gens"], data["accs"], linestyle=lmap[i%len(lmap)], color=cmap[j%len(cmap)],
                         label= m_name + " (" + str(v_accs[b][j]) + "%)")

        plt.legend(bbox_to_anchor=(1, 1), loc='upper left') 
        plt.xlabel("Generation")
        plt.ylabel("Accuracy (%)")
        plt.show()

    else:
        data = get_metric(base,smooth,Metrics,metric)
        sns.heatmap(data, annot=True, fmt=".2f",vmin=vminmax[0], vmax=vminmax[1], cbar=cbar)
        plt.xlabel("base model")
        plt.ylabel("smoother")
        plt.show()
        
    return data

In [ ]:
base = ["rf", "lgb", "xgb", "logreg"]
smooth = ["cnv", "xgb", "crf"]

cols = ["base", "smooth","data","gen","bal","snp","val_acc"]
ACC = [] # (b, s, data, gen, bal, ukb, acc)
all_accs = np.zeros( (len(base), len(smooth), len(gen_expts), len(datasets_benchmark)) )
for g, gen_name in enumerate(gen_expts):
    for d, data_path in enumerate(datasets_benchmark):
        bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
        metric_path = bm_root + "benchmark_results.pkl"
        M = load_dict(metric_path)
        
        v_accs = get_metric(base, smooth, M, "val_acc")
        
        dat = np.array(bm_root.split("/"))[-5]
        bal = np.array(bm_root.split("/"))[-6]
        ukb = np.array(bm_root.split("/"))[-7]
        for b in base:
            for s in smooth:
                ACC += [(b,s,dat,gen_name,bal,ukb,v_accs[b][s])]
DF = pd.DataFrame(ACC, columns=cols)
DF

## (LogReg) X (XGB, CRF) performing best

In [ ]:
DF.boxplot(column=["val_acc"], by=["snp","base", "smooth"], figsize=(16,8), rot="45")
plt.show()

In [ ]:
DF.boxplot(column=["val_acc"], by=["snp", "base", "smooth"], figsize=(16,8), rot="45")
plt.show()

## Closer look

In [ ]:
DF_best_models = DF[ 
    ~(DF.base.isin(["rf", "lgb"])) &
    (DF.smooth != "cnv")
]
DF_best_models.boxplot(column=["val_acc"], by=["base", "smooth"], figsize=(16,8), rot="45")
plt.show()

## Balance not affacting much but still a bit
we want to repeat with fixed validation set

In [ ]:
DF.boxplot(column=["val_acc"], by=["data", "bal"], figsize=(16,8), rot=45)
plt.show()

## same pattern using only best models

In [ ]:
DF_best_models.boxplot(column=["val_acc"], by=["data", "bal"], figsize=(16,8), rot=45)
plt.show()

## Accuracies similar for high generations

In [ ]:
DF_lathi = DF[ 
    (DF.base != "rf") &
    (DF.smooth != "cnv") &
    (DF.gen=="high") ]
DF_lathi.boxplot(column=["val_acc"], by=["data", "snp", "smooth"], figsize=(16,8), rot=45)
plt.show()

In [ ]:
data_path = "/home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/five/"
base = ["xgb", "logreg"]
smooth = ["xgb", "crf"]
gen_name = "high"
bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
metric_path = bm_root + "benchmark_results.pkl"
M = load_dict(metric_path)
out = plt_metric(base ,smooth, M, "gen_performance")

## Note! acc > 90% for gen 100